In [177]:
import os
import pandas as pd
import numpy as np
from datetime import date
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [178]:
print_to_excel = True

In [179]:
excel_data = pd.read_excel('report_invest_30.05.2024.xlsx', sheet_name='snapshot', header=10)

In [228]:
# excel_data_pivot = excel_data.pivot_table(index=['Балансовая единица','Завод','ПЕ','СПП-элемент','Контрагент','Вид проекта','Профиль проекта','Проект','Вид затрат','Версия'],values='Заказ на поставку', aggfunc=sum).reset_index() #group data
excel_data_work = excel_data
excel_data_work['holding'] = merge_SalesUnits(excel_data_work,'Балансовая единица',merge_col='holding') # merge holding
# create version columns
new_cols = ['Версия_045', 'Версия_075', 'Версия_035', 'Версия_085']
excel_data_work[new_cols] = None
for i in excel_data_work.index:
    excel_data_work.loc[i, [x for x in new_cols if str(excel_data_work.loc[i,'Версия']) in x][0]] = excel_data_work.loc[i,'Заказ на поставку']
excel_data_version_pivot = excel_data_work.pivot_table(index=['Балансовая единица','Завод',
       'ПЕ','СПП-элемент','Контрагент','Вид проекта','Профиль проекта',
       'Проект','Вид затрат','КалендГод/Месяц', 'Календарный год',
       'Календарный месяц', 'ПВК|Тип прогноз. данных',
       'holding'], values=new_cols, aggfunc=sum).reset_index().sort_values(['Балансовая единица','Контрагент'])
# compaire values
excel_data_version_pivot['amount'] = None
# conditions
# Если сумма платежей (значение поля " версия " = 075) равна сумме освоения (значение поля " версия " = 035 для БЕ из holding = SUEK; либо " версия " = 085 для БЕ из holding = Eurochem), то берем платежи с НДС (значение поля " версия " = 045)
excel_data_version_pivot.loc[((excel_data_version_pivot.Версия_035==excel_data_version_pivot.Версия_075)&(excel_data_version_pivot.holding=='SUEK'))|((excel_data_version_pivot.Версия_085==excel_data_version_pivot.Версия_075)&(excel_data_version_pivot.holding!='SUEK')&(excel_data_version_pivot.holding!='External')), 'amount'] = excel_data_version_pivot.loc[((excel_data_version_pivot.Версия_035==excel_data_version_pivot.Версия_075)&(excel_data_version_pivot.holding=='SUEK'))|((excel_data_version_pivot.Версия_085==excel_data_version_pivot.Версия_075)&(excel_data_version_pivot.holding!='SUEK')&(excel_data_version_pivot.holding!='External')), 'Версия_045']
# Если сумма платежей (значение поля " версия " = 075) меньше суммы освоения (значение поля " версия " = 035 для БЕ из holding = SUEK; либо " версия " = 085 для БЕ из holding = Eurochem), то берем платежи с НДС (значение поля " версия " = 045)
excel_data_version_pivot.loc[((excel_data_version_pivot.Версия_075<excel_data_version_pivot.Версия_035)&(excel_data_version_pivot.holding=='SUEK'))|((excel_data_version_pivot.Версия_075<excel_data_version_pivot.Версия_085)&(excel_data_version_pivot.holding!='SUEK')&(excel_data_version_pivot.holding!='External')), 'amount'] = excel_data_version_pivot.loc[((excel_data_version_pivot.Версия_075<excel_data_version_pivot.Версия_035)&(excel_data_version_pivot.holding=='SUEK'))|((excel_data_version_pivot.Версия_075<excel_data_version_pivot.Версия_085)&(excel_data_version_pivot.holding!='SUEK')&(excel_data_version_pivot.holding!='External')), 'Версия_045']
# Если сумма платежей (значение поля " версия " = 075) больше суммы освоения (значение поля " версия " = 035 для БЕ из holding = SUEK; либо " версия " = 085 для БЕ из holding = Eurochem), то берем сумму освоения (035 или 085) и равномерно распределяем по срокам, соответствующим будущим платежам (075), начиная с самого дальнего. Т.к. объём освоения меньше, чем платежей, то таким образом будет "заполнена" только какая-то часть платежей. В итоге в unifiedOCP надо забрать платежи c НДС (045), взятые в той же пропорции, в которой были заполнены платежи без НДС (075)
# part data
part_data = excel_data_version_pivot[((excel_data_version_pivot.Версия_075>excel_data_version_pivot.Версия_035)&(excel_data_version_pivot.holding=='SUEK'))]
for version_col in new_cols:
    part_data.loc[part_data[version_col]==0,version_col]=np.NaN
part_data_pivot = part_data.pivot_table(index=['КалендГод/Месяц','ПВК|Тип прогноз. данных'],values=new_cols,aggfunc=['count','sum']).reset_index()
# create dict
unique_ccy_list = part_data_pivot['ПВК|Тип прогноз. данных'].unique()
ccy_number_045_dict = {}
for ccy in unique_ccy_list:
    part_data_pivot_ccy = part_data_pivot[part_data_pivot['ПВК|Тип прогноз. данных']==ccy]
    percent = part_data_pivot_ccy['count']['Версия_035'].sum()/part_data_pivot_ccy['count']['Версия_075'].sum()
    number_of_045_sum = part_data_pivot_ccy['count']['Версия_045'].sum()
    number_of_045_part = int(number_of_045_sum*percent)
    if number_of_045_part>0:
        ccy_number_045_dict[ccy] = [number_of_045_sum-number_of_045_part, part_data_pivot_ccy, number_of_045_sum]
# full data
for not_null_ccy in list(ccy_number_045_dict.keys()):
    part_data[(part_data['ПВК|Тип прогноз. данных']==not_null_ccy)&(~part_data['Версия_045'].isna())].reset_index(drop=True).loc[ccy_number_045_dict[not_null_ccy][0]:,:]['amount'] = part_data[(part_data['ПВК|Тип прогноз. данных']==not_null_ccy)&(~part_data['Версия_045'].isna())].reset_index(drop=True).loc[ccy_number_045_dict[not_null_ccy][0]:,:]['Версия_045']
excel_data_version_pivot[((excel_data_version_pivot.Версия_075>excel_data_version_pivot.Версия_035)&(excel_data_version_pivot.holding=='SUEK'))] = part_data

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SUEK' 'SUEK' 'SUEK' ... 'SUEK' 'SUEK' 'SUEK']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10976\1562378557.py:9: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future versio

In [229]:
# compaire with old BD
old_BD_data_SUEK = pd.read_excel('2024-06-13_SUEK_Invest.xlsx',header=3).drop(0,axis=0) 
old_BD_data_SUEK_pivot = old_BD_data_SUEK.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
old_BD_data_EURO = pd.read_excel('2024-06-13_EUROCHEM_Invest.xlsx',header=3).drop(0,axis=0) 
old_BD_data_EURO_pivot = old_BD_data_EURO.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
old_BD_data_SUEK_pivot

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10976\4027979718.py:3: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  old_BD_data_SUEK_pivot = old_BD_data_SUEK.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10976\4027979718.py:5: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  old_BD_data_EURO_pivot = old_BD_data_EURO.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Зак

,БЕ,Сумма_итого_законтрактровнано
0,1100,156069493.09
1,1300,72101.36
2,1600,19216.02
3,1I00,104629.4
4,1M00,1584353.68
...,...,...
76,Y100,155526600.61
77,Y300,18343444.65
78,Y500,26009.73
79,Y600,0


In [241]:
compaire_to_print = excel_data_version_pivot.pivot_table(index='Балансовая единица',aggfunc=sum,values='amount').reset_index().merge(old_BD_data_SUEK_pivot,how='outer',left_on='Балансовая единица',right_on='БЕ').sort_values('amount',ascending=False)
compaire_to_print = compaire_to_print.fillna(0)
compaire_to_print['is_equal'] = compaire_to_print.amount.astype(int)==compaire_to_print['Сумма_итого_законтрактровнано'].astype(int)
if print_to_excel==True:
    Output_file = f'{str(date.today())}_Invest_from_report_invest_30.05.2024.xlsx'
    compaire_to_print.to_excel(Output_file,sheet_name='compaire',index=False)
    new_list(excel_data_version_pivot,Output_file,'data')

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10976\3206749837.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  compaire_to_print = excel_data_version_pivot.pivot_table(index='Балансовая единица',aggfunc=sum,values='amount').reset_index().merge(old_BD_data_SUEK_pivot,how='outer',left_on='Балансовая единица',right_on='БЕ').sort_values('amount',ascending=False)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10976\3206749837.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  compaire_to_print = compaire_to_print.fillna(0)


In [242]:
compaire_to_print

,Балансовая единица,amount,БЕ,Сумма_итого_законтрактровнано,is_equal
90,U100,2.089641e+10,U100,4.002332e+08,False
94,U600,1.931697e+10,U600,1.494691e+06,False
108,Y100,1.313508e+10,Y100,1.555266e+08,False
0,1100,8.405355e+09,1100,1.560695e+08,False
42,6700,6.151916e+09,6700,1.078138e+08,False
...,...,...,...,...,...
41,0,0.000000e+00,6600,0.000000e+00,True
45,0,0.000000e+00,6B00,0.000000e+00,True
54,0,0.000000e+00,9G00,0.000000e+00,True
88,0,0.000000e+00,Q100,0.000000e+00,True
